2433

In [1]:
!pip install transformers trl torch peft accelerate

In [2]:
! pip install -U "huggingface_hub[cli]"

In terminal: `huggingface-cli login`

In [3]:
import os
import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from trl import SFTTrainer
import gc

In [4]:
checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"
device = torch.device("cuda")
tokenizer = AutoTokenizer.from_pretrained(checkpoint, map_device="cuda")
tokenizer.pad_token_id = tokenizer.bos_token_id
dataset = load_dataset("json", data_files="ft_dataset.jsonl", split="train" )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", is_decoder=True)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
    
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    tokenizer=tokenizer,
    dataset_batch_size=10,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/user/miniconda/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
trainer.train()
trainer.save_model("output_dir")

Step,Training Loss


In [9]:
finetuned_model = AutoModelForCausalLM.from_pretrained("./output_dir", is_decoder=True, device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
text = """
    Uninsured deposits had nothing to do with SVB's collapse. It's all about poor management and risky investments. Stop blaming the victims!
    Oh, really? So you're saying if all deposits were insured, depositors wouldn't have rushed to withdraw? Isn't that the very definition of a bank run?"
    "That's not what I said. I'm saying the bank's poor decisions led to panic, not the lack of insurance. But sure, twist my words."
    """
inputs = tokenizer(text, return_tensors="pt") 

with torch.no_grad(): 
    outputs = finetuned_model(**inputs)
    
generated_tokens = finetuned_model.generate(**inputs, max_new_tokens=100) 
print("Output:\n" + 100 * '-')
print(tokenizer.decode(generated_tokens[0], skip_special_tokens=True))    

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------

    Uninsured deposits had nothing to do with SVB's collapse. It's all about poor management and risky investments. Stop blaming the victims!
    Oh, really? So you're saying if all deposits were insured, depositors wouldn't have rushed to withdraw? Isn't that the very definition of a bank run?"
    "That's not what I said. I'm saying the bank's poor decisions led to panic, not the lack of insurance. But sure, twist my words."
     "I'm not twisting your words. I'm pointing out that you're downplaying the role of uninsured deposits in the bank's collapse. It's a complex issue, but it's clear that the lack of insurance created a sense of urgency that exacerbated the crisis."
    "Ugh, fine. I see what you're saying now. But it's still not the whole story. The bank's management was reckless, and that's what ultimately led to its downfall. We can't just blame the
